References: https://ragntune.com/blog/guide-fine-tuning-code-llama?source=post_page-----7d40e6ad33b4--------------------------------

In [1]:
!pip install git+https://github.com/huggingface/transformers.git@main bitsandbytes  # we need latest transformers for this
!pip install git+https://github.com/huggingface/peft.git@4c611f4
!pip install datasets==2.11
!pip install wandb
!pip install scipy

  Cloning https://github.com/huggingface/transformers.git (to revision main) to /tmp/pip-req-build-j2nsjpv5
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-j2nsjpv5
  Resolved https://github.com/huggingface/transformers.git to commit 03732dea60fba1da78c79eb59c443ebf975c2be6
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 67.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 65.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 97.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [2]:
! pip install codebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 22.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached tree_sitter-0.21.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (496 kB)
  Created wheel for codebleu: filename=codebleu-0.6.0-cp310-cp310-linux_x86_64.whl size=1281126 sha256=820a38fa69065e0a7da0b6c8310bf1c86dcb653d6379f8684360544b2c26a9c0
  Stored in directory: /root/.cache/pip/wheels/5e/45/f6/abb2f927620de36cc631f769fc8f478fe44f7e057dc3b09483
Successfully built codebleu


In [3]:
from datetime import datetime
import os
import sys

import torch
from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_int8_training,
    set_peft_model_state_dict,
)
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForSeq2Seq



In [5]:
from datasets import load_dataset
dataset = load_dataset("csv", data_files='./data/train.csv')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-c8bb6f246e98b3ee/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
train_dataset = dataset["train"].train_test_split(test_size=0.1)["train"]
eval_dataset = dataset["train"].train_test_split(test_size=0.1)["test"]

In [7]:
base_model = "codellama/CodeLlama-7b-Instruct-hf"
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained("codellama/CodeLlama-7b-hf")

config.json:   0%|          | 0.00/646 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
# Example prompt for SQL

eval_prompt = """You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.

You must output the SQL query that answers the question.
### Input:
Which Class has a Frequency MHz larger than 91.5, and a City of license of hyannis, nebraska?

### Context:
CREATE TABLE table_name_12 (class VARCHAR, frequency_mhz VARCHAR, city_of_license VARCHAR)

### Response:
"""

model_input = tokenizer(eval_prompt, return_tensors="pt").to(device)

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=512)[0], skip_special_tokens=True))


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.

You must output the SQL query that answers the question.
### Input:
Which Class has a Frequency MHz larger than 91.5, and a City of license of hyannis, nebraska?

### Context:
CREATE TABLE table_name_12 (class VARCHAR, frequency_mhz VARCHAR, city_of_license VARCHAR)

### Response:
SELECT * FROM table_name_12 WHERE frequency_mhz > 91.5 AND city_of_license = 'hyannis, nebraska'

### Input:
Which Class has a Frequency MHz larger than 91.5, and a City of license of hyannis, nebraska?

### Context:
CREATE TABLE table_name_12 (class VARCHAR, frequency_mhz VARCHAR, city_of_license VARCHAR)

### Response:
SELECT * FROM table_name_12 WHERE frequency_mhz > 91.5 AND city_of_license = 'hyannis, nebraska'

### Input:
Which Class has a Frequency MHz larger than 91.5, and a City of license of hyannis, nebraska?

### Context:
CREATE TABLE table_nam

In [9]:
tokenizer.add_eos_token = True
tokenizer.pad_token_id = 0
tokenizer.padding_side = "left"

In [10]:
def tokenize(prompt):
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=512,
        padding=False,
        return_tensors=None,
    )

    # "self-supervised learning" means the labels are also the inputs:
    result["labels"] = result["input_ids"].copy()

    return result

In [11]:
def generate_and_tokenize_prompt(data_point):
    full_prompt =f"""You are a helpful, precise, detailed, and concise artificial intelligence
 assistant with a deep expertise in Javascript and Typescript, specifically for the React
 library.

 You are very intelligent, having keen ability to discern the essentials
 of word problems, which you are asked to implement in software.

 In this task, you are asked to read the problem description and return
 the software code that satisfies the prompt.  Please output
 only the code; do not include any explanations, justifications, or
 auxiliary content.

 You will be evaluated based on the following criteria:
 - Generated answer is a piece of software that can run without errors.
 - The generated answer is the cleanest and most efficient implementation.

 Below is a programming prompt. Your task is to complete the implementation
 by adding the necessary code that offers the best possible solution to
 the stated prompt.

 ### Prompt: {data_point["prompt"]}

 ### Solution: {data_point["output"]}
  """
    return tokenize(full_prompt)


In [12]:
tokenized_train_dataset = train_dataset.map(generate_and_tokenize_prompt)
tokenized_val_dataset = eval_dataset.map(generate_and_tokenize_prompt)

Map:   0%|          | 0/1350 [00:00<?, ? examples/s]

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

In [13]:
test_data = load_dataset("csv", data_files='./data/test.csv')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-9919b12627fb369e/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [14]:
test_data

DatasetDict({
    train: Dataset({
        features: ['output', 'prompt'],
        num_rows: 50
    })
})

In [15]:
from codebleu import calc_codebleu
from tqdm import tqdm

def evaluate_model(model):

  test_data = load_dataset("csv", data_files='./data/test.csv')
  model.eval()

  total_score = 0.0
  max_score = 0.0
  for sample in tqdm(test_data["train"]):
    eval_prompt = f"""You are a helpful, precise, detailed, and concise artificial intelligence
      assistant with a deep expertise in Javascript and Typescript, specifically for the React
      library.

      You are very intelligent, having keen ability to discern the essentials
      of word problems, which you are asked to implement in software.

      In this task, you are asked to read the problem description and return
      the software code that satisfies the prompt.  Please output
      only the code; do not include any explanations, justifications, or
      auxiliary content.

      You will be evaluated based on the following criteria:
      - Generated answer is a piece of software that can run without errors.
      - The generated answer is the cleanest and most efficient implementation.

      Below is a programming prompt. Your task is to complete the implementation
      by adding the necessary code that offers the best possible solution to
      the stated prompt.

      ### Prompt: {sample["prompt"]}

      ### Solution:
        """

    model_input = tokenizer(eval_prompt, return_tensors="pt").to(device)
    with torch.no_grad():
      output = tokenizer.decode(model.generate(**model_input, max_new_tokens=512)[0],
                                skip_special_tokens=True)
      expected_output = sample["output"]

      score = calc_codebleu([output],
                            [expected_output],
                            lang="javascript",
                            weights=(0.33, 0.34, 0.33, 0),
                            tokenizer=None)['codebleu']
      total_score += score

      if score > max_score:
        max_score = score

  return total_score / len(test_data["train"]), max_score

In [17]:
evaluate_model(model)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
  0%|          | 0/50 [00:01<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
model.train() # put model back into training mode
model = prepare_model_for_int8_training(model)

# Used rank 8 for LoRA
config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=[
    "q_proj",
    "k_proj",
    "v_proj",
    "o_proj",
],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, config)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:133: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


In [ ]:
batch_size = 128
per_device_train_batch_size = 32
gradient_accumulation_steps = batch_size // per_device_train_batch_size
output_dir = "react-code-llama-7b"

training_args = TrainingArguments(
        per_device_train_batch_size=per_device_train_batch_size,
        gradient_accumulation_steps=gradient_accumulation_steps,
        warmup_steps=100,
        max_steps=20,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=5,
        optim="adamw_torch",
        lr_scheduler="cosine",
        evaluation_strategy="steps", # if val_set_size > 0 else "no",
        save_strategy="steps",
        eval_steps=10,
        save_steps=10,
        output_dir=output_dir,
        load_best_model_at_end=False,
        group_by_length=True, # group sequences of roughly the same length together to speed up training
        report_to="wandb", # if use_wandb else "none",
        run_name=f"codellama-{datetime.now().strftime('%Y-%m-%d-%H-%M')}", # if use_wandb else None,
    )

trainer = Trainer(
    model=model,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    args=training_args,
    data_collator=DataCollatorForSeq2Seq(
        tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True
    ),
)

In [ ]:
trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


OutOfMemoryError: CUDA out of memory. Tried to allocate 688.00 MiB. GPU 0 has a total capacity of 22.17 GiB of which 152.88 MiB is free. Process 37325 has 22.01 GiB memory in use. Of the allocated memory 18.73 GiB is allocated by PyTorch, and 3.05 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
peft_model_id = "react-code-llama-7b-trained"
model.save_pretrained(peft_model_id)